### Build ResNet50

In [ ]:
# refer to keras.applications.resnet

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import warnings

from tensorflow.keras import backend, layers, models
import keras.utils as keras_utils
from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import decode_predictions,preprocess_input

'''
WEIGHTS_PATH : https://github.com/fchollet/deep-learning-models/
                releases/download/v0.2/
                resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP : https://github.com/fchollet/deep-learning-models/
                       releases/download/v0.2/'
                       resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
'''

global backend, layers, models, keras_utils

def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x


def ResNet50(input_shape=None,
             pooling=None,
             classes=100,
             **kwargs):

    global backend, layers, models, keras_utils
    
    # the input of ImageNet pre-train model sizes 224 * 224 * 224
    # Upsample images in Cifar100 (32, 32, 3) to (256, 256, 3) first.
    # Set input shape to (256,256,3)
    img_input = layers.Input(shape=(256,256,3))

    # the axis of color（channel）is 3
    bn_axis = 3

    # connect the blocks
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')


    inputs = img_input

    # Create model.
    base_model = models.Model(inputs, x, name='resnet50')

    # Load weights.
    weights_path = keras_utils.get_file(
            'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
            WEIGHTS_PATH_NO_TOP,
            cache_subdir='models',
            md5_hash='a268eb855778b3df3c7506639542a6af')
    base_model.load_weights(weights_path)
    for layer in base_model.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False    
    input_layer = layers.Input(shape=input_shape)
    x = layers.UpSampling2D()(input_layer)
    x = layers.UpSampling2D()(x)
    x = layers.UpSampling2D()(x)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(.25)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(classes, activation='softmax', name='fc_classes')(x)
    # this is the model we will train
    # Create another model with softmax
    model = models.Model(inputs=input_layer, outputs=x)

    return model


Using TensorFlow backend.


### Train Data

In [ ]:
import numpy as np

# data augmentation function
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
img_rows, img_cols = 32, 32 # Resolution of original inputs
channel = 3
num_classes = 100 
batch_size = 16 
nb_epoch = 100

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar100 
from keras.preprocessing.image import ImageDataGenerator

(X_train, Y_train), (X_valid, Y_valid) = cifar100.load_data()
nb_train_samples = len(X_train)
nb_valid_samples = len(X_valid)

print(X_train.shape)

#Pre-process the data
X_train = preprocess_input(X_train)
X_valid = preprocess_input(X_valid)

datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
datagen.fit(X_train)

Y_train = keras_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
Y_valid = keras_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)
print(Y_train.shape)

(50000, 32, 32, 3)
(50000, 100)


In [ ]:
import time
import tensorflow as tf

# Model parallel
# But I am not sure it turns out successful
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    model = ResNet50(classes = num_classes, input_shape=(img_rows, img_cols, channel))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
history = model.fit_generator(datagen.flow(X_train, Y_train,
                                  batch_size=64),
                                  steps_per_epoch=X_train.shape[0] // 64,
                                  epochs=100,
                                  validation_data=(X_valid, Y_valid))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')
Number of devices: 5
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/dev

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



781/781 [==============================] - 78s 99ms/step - loss: 0.7546 - accuracy: 0.7778 - val_loss: 0.7122 - val_accuracy: 0.8027
Epoch 26/100
781/781 [==============================] - 77s 98ms/step - loss: 0.7365 - accuracy: 0.7796 - val_loss: 0.6816 - val_accuracy: 0.8039
Epoch 27/100
781/781 [==============================] - 77s 99ms/step - loss: 0.7332 - accuracy: 0.7819 - val_loss: 0.7057 - val_accuracy: 0.8030
Epoch 28/100
781/781 [==============================] - 78s 99ms/step - loss: 0.7148 - accuracy: 0.7870 - val_loss: 0.6937 - val_accuracy: 0.8051
Epoch 29/100
781/781 [==============================] - 77s 99ms/step - loss: 0.7179 - accuracy: 0.7871 - val_loss: 0.6919 - val_accuracy: 0.8029
Epoch 30/100
781/781 [==============================] - 77s 99ms/step - loss: 0.7065 - accuracy: 0.7886 - val_loss: 0.6916 - val_accuracy: 0.8050
Epoch 31/100
781/781 [==============================] - 77s 99ms/step - loss: 0.6973 - accuracy: 0.7905 - val_loss: 0.6886 - val_accuracy